#### **Reporte Caso 5** ####
#### **Problema de Cartera de Inversión** ####
#### **Asignatura:** Programación para Analítica Prescriptiva y de Apoyo a la Decisión ####
#### **Instructor:** Dr. Josué Domínguez Guerrero ####
#### **Alumno:** Aminadab Córdova Acosta ####

**1. Definición del problema**

En el ámbito financiero, la asignación óptima de capital entre múltiples activos es un problema fundamental. El modelo clásico de optimización de portafolio de Markowitz busca encontrar una combinación de activos que maximice el rendimiento esperado ajustado por el riesgo. En este problema, se plantea un escenario con 20 activos financieros, cada uno con un rendimiento esperado y una relación de covarianza con el resto del portafolio. El objetivo es maximizar el rendimiento neto, penalizando el riesgo total del portafolio medido mediante su varianza. La no linealidad del modelo proviene de esta penalización cuadrática por riesgo, la cual depende de la interacción entre todas las proporciones de inversión.

---

**2. Modelo Matemático**

Variables

- \( w_i \): fracción del capital total invertida en el activo \( i \)

---

Función Objetivo

Maximizar el rendimiento esperado penalizado por el riesgo:

$$
\max \left( \sum_i \mu_i w_i - \lambda \sum_i \sum_j w_i \cdot \Sigma_{ij} \cdot w_j \right)
$$

donde:

- \( \mu_i \): rendimiento esperado del activo \( i \)  
- \( \Sigma_{ij} \): covarianza entre los activos \( i \) y \( j \)  
- \( \lambda \): coeficiente de aversión al riesgo

---

Restricciones

1. Presupuesto total:

$$
\sum_i w_i = 1
$$

2. No se permite venta corta:

$$
w_i \geq 0 \quad \forall i
$$

**3. Implementación en Pyomo**

In [ ]:
import pandas as pd
import numpy as np
from pyomo.environ import *

# Leer la matriz de covarianza
df_cov = pd.read_csv(r"D:\AminadabCA\MIAAD\Analítica_Prescriptiva\Ejercicios_casos\Matriz_Covarianza_Portafolio.csv", index_col=0)

# Simular rendimientos esperados (entre 6% y 12%)
np.random.seed(42)
expected_returns = np.random.uniform(0.06, 0.12, len(df_cov.columns))
assets = list(df_cov.columns)
returns_dict = dict(zip(assets, expected_returns))

# Crear modelo Pyomo
model = ConcreteModel()

model.Assets = Set(initialize=assets)
model.mu = Param(model.Assets, initialize=returns_dict)
model.Sigma = Param(model.Assets, model.Assets,
                    initialize=lambda model, i, j: df_cov.loc[i, j])
lambda_risk = 0.5

model.x = Var(model.Assets, domain=NonNegativeReals)

# Restricción: toda la inversión se asigna
model.budget_constraint = Constraint(expr=sum(model.x[i] for i in model.Assets) == 1)

# Objetivo: Maximizar utilidad neta (rendimiento menos riesgo)
def objective_rule(model):
    retorno = sum(model.mu[i] * model.x[i] for i in model.Assets)
    riesgo = sum(model.Sigma[i, j] * model.x[i] * model.x[j]
                 for i in model.Assets for j in model.Assets)
    return retorno - lambda_risk * riesgo

model.obj = Objective(rule=objective_rule, sense=maximize)

# Resolver con IPOPT o el solver que tengas disponible
solver = SolverFactory('ipopt')  # 
results = solver.solve(model, tee=True)

# Mostrar resultados
for i in model.Assets:
    print(f"{i}: {model.x[i].value:.4f} (rend: {returns_dict[i]:.2%})")


Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

**4. Interpretación de la decisión preescrita por el modelo de decisión**

El modelo prescribe invertir 41.7% en el activo A2 y 58.3% en el activo A8, excluyendo al resto. Esta decisión maximiza el rendimiento esperado del portafolio ajustado por riesgo, priorizando los activos con mejor relación rendimiento-riesgo según la covarianza entre ellos.